# IBMs Applied Data Science Capstone Week 3 Assignment

## Segmenting and Clustering Neighborhoods in Toronto

### Libraries Loading:

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Odyskypa\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1h             |       he774522_0         5.8 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:       

### Wikipedia scraping using Beautiful Soup:

In [5]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

### Obtaining the data that is in the table of postal codes and transforming the data into a pandas dataframe.

In [9]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Remove "Not assigned" values from "Borough" column.

In [10]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### More than one neighborhood can exist in one postal code area. These examples will be combined into one row with the neighborhoods separated with a comma.

In [11]:
df_new = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_new.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_new

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
df_new.shape

(103, 3)

### Loading Geospatial Data CSV.

In [13]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

### Merging Geospatial Data with df_new on "Postcode" and Creating Toronto's Dataframe.

In [15]:
df_merged = pd.merge(df_new, df_geo, on=['Postcode'], how='inner')

df_toronto = df_merged[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_toronto.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Finding Toronto's Latitude and Longtitude.

In [17]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
# Create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

# Neighbourhood Clustering based on Top Common Venues.

### Define Foursquare Credentials.

In [20]:
CLIENT_ID = 'ZBR40AO52XV1C2ICNIUKUNX5E1ENBNQ54JCLOWKNX0FCVCZ4' # your Foursquare ID
CLIENT_SECRET = 'P2SDPTGVAVLDDQGXAEL2LHAZVPYNJAWMZ5LQ2Y3UJ0GKITNZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZBR40AO52XV1C2ICNIUKUNX5E1ENBNQ54JCLOWKNX0FCVCZ4
CLIENT_SECRET:P2SDPTGVAVLDDQGXAEL2LHAZVPYNJAWMZ5LQ2Y3UJ0GKITNZ


### Let's explore Top 100 venues in each neighbourhood within 500 meters radius.

In [22]:
# define limit = 5 (limit to 5 venues only) & radius = 500 (meters)
LIMIT = 100
radius = 500

location_list = [] # initiate a list to store data from Foursquare API requests

for neighbourhood, latitude, longitude in zip(df_toronto['Neighbourhood'], df_toronto['Latitude'], df_toronto['Longitude']):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json()
    
    # use len() to check if any data within "items" (len = 0 if nothing)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue # skip the row if nothing is found
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # extract info within 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        location_list.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])

### Create a dataframe consisting of "location_list" data .

In [34]:
toronto_venues = pd.DataFrame(x for row in location_list for x in row)
toronto_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
4,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant


### One - Hot Encoding to prepare dataframe for clustering.

In [38]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Field,Fish & Chips Shop,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Martial Arts School,Massage Studio,Motel,Museum,Neighborhood,Park,Pharmacy,Pizza Place,Playground,Pool,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Sports Bar,Theme Restaurant,Trail,Warehouse Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cedarbrae,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_onehot.shape

(100, 59)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Field,Fish & Chips Shop,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Martial Arts School,Massage Studio,Motel,Museum,Park,Pharmacy,Pizza Place,Playground,Pool,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Sports Bar,Theme Restaurant,Trail,Warehouse Store
0,Agincourt,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Birch Cliff, Cliffside West",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"Business reply mail Processing Centre, South C...",0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,1.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
toronto_grouped.shape


(96, 59)

### Let's print each neighborhood along with the top 5 most common venues.

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Breakfast Spot   1.0
1     Yoga Studio   0.0
2        Pharmacy   0.0
3    Hockey Arena   0.0
4           Hotel   0.0


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place   1.0
1   Yoga Studio   0.0
2          Park   0.0
3           Gym   0.0
4  Hockey Arena   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0  Deli / Bodega   1.0
1    Yoga Studio   0.0
2           Park   0.0
3   Hockey Arena   0.0
4          Hotel   0.0


----Bayview Village----
                venue  freq
0  Chinese Restaurant   1.0
1         Yoga Studio   0.0
2            Pharmacy   0.0
3        Hockey Arena   0.0
4               Hotel   0.0


----Bedford Park, Lawrence Manor East----
          venue  freq
0          Café   1.0
1   Yoga Studio   0.0
2      Pharmacy   0.0
3  Hockey Arena   0.0
4         Hotel   0.0


----Berczy Park----
          venue  freq
0    Restaurant   1.0
1   Yoga Studio   0.0
2          Park   0

4  Hockey Arena   0.0


----Studio District----
            venue  freq
0  Ice Cream Shop   1.0
1     Yoga Studio   0.0
2            Park   0.0
3             Gym   0.0
4    Hockey Arena   0.0


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
          venue  freq
0  Liquor Store   1.0
1   Yoga Studio   0.0
2          Park   0.0
3           Gym   0.0
4  Hockey Arena   0.0


----The Annex, North Midtown, Yorkville----
          venue  freq
0          Café   1.0
1   Yoga Studio   0.0
2      Pharmacy   0.0
3  Hockey Arena   0.0
4         Hotel   0.0


----The Beaches----
          venue  freq
0         Trail   1.0
1   Yoga Studio   0.0
2          Park   0.0
3           Gym   0.0
4  Hockey Arena   0.0


----The Danforth West, Riverdale----
              venue  freq
0  Greek Restaurant   1.0
1              Park   0.0
2               Gym   0.0
3      Hockey Arena   0.0
4             Hotel   0.0


----The Kingsway, Montgomery Road, Old Mill North----
          venue  

### Let's put that into a pandas dataframe.


In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop
1,"Alderwood, Long Branch",Pizza Place,Warehouse Store,Chinese Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Deli / Bodega,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop
3,Bayview Village,Chinese Restaurant,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop,Department Store
4,"Bedford Park, Lawrence Manor East",Café,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop


### Cluster Neighborhoods.

Run k-means to cluster the neighborhood into 5 clusters.

In [45]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 4, 3, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [69]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.insert(0, 'Latitude', toronto_venues['Neighborhood Latitude'])
#neighborhoods_venues_sorted.insert(0, 'Longitude', toronto_venues['Neighborhood Longitude'])
neighborhoods_venues_sorted.drop(['Latitude'], axis=1)
neighborhoods_venues_sorted.head()

,Latitude,Longitude,Latidute,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.806686,-79.194353,43.806686,3,Agincourt,Breakfast Spot,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop
1,43.784535,-79.160497,43.784535,3,"Alderwood, Long Branch",Pizza Place,Warehouse Store,Chinese Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop,Department Store
2,43.763573,-79.188711,43.763573,3,"Bathurst Manor, Wilson Heights, Downsview North",Deli / Bodega,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop
3,43.770992,-79.216917,43.770992,3,Bayview Village,Chinese Restaurant,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop,Department Store
4,43.773136,-79.239476,43.773136,3,"Bedford Park, Lawrence Manor East",Café,Warehouse Store,Clothing Store,Garden,Fish & Chips Shop,Field,Fast Food Restaurant,Dog Run,Diner,Dessert Shop


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters